In [ ]:
!pip install tqdm
!pip install nltk
!pip install wordcloud

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from string import punctuation
import nltk
import os

import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
from bs4 import BeautifulSoup


from wordcloud import WordCloud, STOPWORDS

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
pd.options.display.max_colwidth = None

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
DATA_PATH = '/kaggle/input/jigsaw-toxic-severity-rating'

In [ ]:
df = pd.read_csv(DATA_PATH + '/validation_data.csv')
# df = pd.read_csv(DATA_PATH + '/comments_to_score.csv')
print(df.columns)
print(df.shape)

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
target_cols = df.columns[1:]
target_cols

In [ ]:
df_sub = pd.read_csv(DATA_PATH + '/comments_to_score.csv')
df_sub.head()

# Text Processing

- Lower Characters
- Regex
- Remove Numbers, Emojis, Punctuation, White Spaces

In [ ]:
# def text_cleaning(text, cols_name):
    
#     df[cols_name] = df[cols_name].apply(lambda x: x.lower())
    
#     return text

In [ ]:
# df_test['less_toxic'] = df_test['less_toxic'].progress_apply(text_cleaning, cols_name='less_toxic')
# df_test['more_toxic'] = df_test['more_toxic'].progress_apply(text_cleaning, cols_name='more_toxic')

df['less_toxic'] = df['less_toxic'].apply(lambda x: x.lower())
df['more_toxic'] = df['more_toxic'].apply(lambda x: x.lower())

In [ ]:
df.head()

# Decontracted

In [ ]:
def cleaned_text(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    template = re.compile(r'https?://\S+|www\.\S+')
    phrase = template.sub(r'', phrase)
    
    soup = BeautifulSoup(phrase, 'lxml') # removes HTML tags
    only_text = soup.get_text()
    phrase = only_text
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    phrase = emoji_pattern.sub(r'', phrase)
    
    phrase = re.sub(r"[^a-zA-Z\d]", " ", phrase) #Remove special Charecters
    phrase = re.sub(' +', ' ', phrase) #Remove Extra Spaces
    phrase = phrase.strip() # remove spaces at the beginning and at the end of string
    
    return phrase

In [ ]:
df['less_toxic'] = df['less_toxic'].progress_apply(cleaned_text)
df['more_toxic'] = df['more_toxic'].progress_apply(cleaned_text)

In [ ]:
df[5:10]

# Remove Number in desc 

In [ ]:
df['less_toxic'] = df['less_toxic'].progress_apply(lambda x: ''.join(string for string in x if not string.isdigit()))
df['more_toxic'] = df['more_toxic'].progress_apply(lambda x: ''.join(string for string in x if not string.isdigit()))

In [ ]:
df[5:10]

In [ ]:
df.head().T

In [ ]:
df.loc[df['less_toxic'].str.contains('\n', case=False)]

# Sentiment and Polarity 

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
SIA = SentimentIntensityAnalyzer()

In [ ]:
def polarity(x):
    if type(x) == str:
        return SIA.polarity_scores(x)
    else:
        return 1000

In [ ]:
df['less_toxic'][0]

In [ ]:
df['more_toxic'][0]

In [ ]:
polarity(df['less_toxic'][0])

In [ ]:
polarity(df['more_toxic'][0])

In [ ]:
# df_test_a = df_test.copy()

In [ ]:
for col in target_cols:
    df[f'polarity_{col}'] = df[col].progress_apply(polarity)
    
    df[f"negativity_{col}"] = df[f'polarity_{col}'].apply(lambda x: x["neg"])

    df[f"positivity_{col}"] = df[f'polarity_{col}'].apply(lambda x: x["pos"])

    df[f"neutrality_{col}"] = df[f'polarity_{col}'].apply(lambda x: x["neu"])

    df[f"compound_{col}"] = df[f'polarity_{col}'].apply(lambda x: x["compound"])

In [ ]:
df.head().T

### Wordcloud

In [ ]:
results = Counter()

In [ ]:
df['less_toxic'].str.lower().str.split().progress_apply(results.update)
df['more_toxic'].str.lower().str.split().progress_apply(results.update)

In [ ]:
count_words = pd.DataFrame.from_dict(results, orient='index').reset_index()

In [ ]:
count_words.columns = ['words', 'count']

In [ ]:
count_words.sort_values(by='count', ascending=False).head(20)

In [ ]:
df[df['less_toxic'].str.contains('wikipedia')]

In [ ]:
df = df[df['more_toxic'].str.contains('uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubnvvi uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubnvvi uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubnvvi uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfbfkjbnknlkshubnsutybnisueynboiserubnsiunybiosubnioseubnoitybnosiubnosriutbynsoitubnosiubnsoiubni uhbsirtubgyihihlkjngkjbnkgjnbkjfgnbknfgjkbnkfjgnbjkfnjbkfnjbkjbnfkjnbkjnbkjnfb') == False].copy()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(15,15))
text = ' '.join([abstract for abstract in df["less_toxic"]])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False
                      ,width=5000, height=5000
                     ).generate(text)
fig = plt.imshow(wordcloud)
fig.axes.set_title('Common words in comments')
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
text = ' '.join([abstract for abstract in df["more_toxic"]])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False
                      ,width=5000, height=5000
                     ).generate(text)
fig = plt.imshow(wordcloud)
fig.axes.set_title('Common words in comments')
plt.show()

In [ ]:
delete_words = ['wikipedia', 'wiki']

In [ ]:
df['less_toxic'] = df['less_toxic'].progress_apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word not in delete_words]))
df['more_toxic'] = df['more_toxic'].progress_apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word not in delete_words]))

### Number of words 

In [ ]:
def new_len(x):
    if type(x) is str:
        return len(x.split())
    else:
        return 0

In [ ]:
df['num_of_words_less_toxic'] = df['less_toxic'].progress_apply(new_len)
df['num_of_words_more_toxic'] = df['more_toxic'].progress_apply(new_len)

In [ ]:
df.head()

In [ ]:
nums_less_toxic = df.query('num_of_words_less_toxic !=0')['num_of_words_less_toxic']
nums_more_toxic = df.query('num_of_words_more_toxic !=0')['num_of_words_more_toxic']

In [ ]:
nums = nums_less_toxic + nums_more_toxic

In [ ]:
len(nums)

In [ ]:
df.shape

In [ ]:
nums

### Num of Words Less Toxic

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Num of Words Less Toxic')
plt.hist(df['num_of_words_less_toxic'], bins=100)
plt.show()

In [ ]:
df['num_of_words_less_toxic'].value_counts()[:10]

### Num of Words More Toxic

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Num of Words More Toxic')
plt.hist(df['num_of_words_more_toxic'], bins=100)
plt.show()

In [ ]:
df['num_of_words_more_toxic'].value_counts()[:10]

In [ ]:
df[df['num_of_words_less_toxic']==0]

In [ ]:
df[df['num_of_words_more_toxic']==0]

In [ ]:
df.to_csv('clean_data.csv', index=False)